# Dependencies & Read Data

In [2]:
#Imports
import pandas as pd
import requests
import gmaps
import re
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
#Read the CSV & Create complete DF
invalid_geocoded_df = pd.read_csv("invalid_facility_data.csv")
invalid_geocoded_df.head(1)

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.96318,-105.509594,0,0,0.0,0.0,0.0


In [8]:
# create testing dataset
testing_data = invalid_geocoded_df.iloc[0:3,:]
testing_data

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,0,0,0.0,0.0,0.0
1,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163077,-105.098557,0,0,0.0,0.0,0.0
2,FA0003618,CARNICERIA EL CERRITO,936 MAIN ST LONGMONT CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.175377,-105.102119,0,0,0.0,0.0,0.0


# Make the Call

In [19]:
params = {
    "rankby": "distance",
    "keyword": "restaurant",
    "key": g_key
}

#iterate through that dataframe!
for index, row in invalid_geocoded_df.iterrows():
    try: 
        # Get the Name Address from the dataframe
        name = row["facilityName"]
        lat = row["google_lat"]
        long = row["google_long"]
        params["location"] = f"{lat},{long}"
        params["name"] = name

        # Set the base URL to grab JSON data
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # Make request and retrieve the JSON data from the search. 
        search = requests.get(base_url, params=params).json()

        #Set Biz status, Name, Rating, price_level, user_ratings_total variables
        # Set try/except statements to account for missing values
        try: 
            g_status = search["results"][0]["business_status"] 
        except (KeyError):
            print(f"{name} not found, skip - Status") 
        try: 
            g_name = search["results"][0]["name"]
        except (KeyError):
            print(f"{name} not found, skip - Name")
        try: 
            g_rating = search["results"][0]["rating"] 
        except (KeyError):
            print(f"{name} not found, skip - Rating")   
        try: 
            g_user_ratings = search["results"][0]["user_ratings_total"]
        except (KeyError):
            print(f"{name} not found, skip - Total User Ratings") 
        try: 
            g_price_level = search["results"][0]["price_level"]
        except (KeyError):
            print(f"{name} not found, skip - Price Level")

        # Get all fields and store
        # Plus try/excepts to skip missing data
        try:
            invalid_geocoded_df.loc[index, "biz_status"] = g_status
        except (NameError):
            continue 
        try:
            invalid_geocoded_df.loc[index, "gbiz_name"] = g_name
        except (NameError):
            continue 
        try:
            invalid_geocoded_df.loc[index, "rating"] = g_rating
        except (NameError):
            continue 
        try:
            invalid_geocoded_df.loc[index, "total_user_ratings"] = g_user_ratings
        except (NameError):
            continue 
        try:
            invalid_geocoded_df.loc[index, "price_level"] = g_price_level
        except (NameError):
            continue
    except (IndexError):
         print(f"{name} not found, skip - Index") 
invalid_geocoded_df

ROASTED TOAD BBQ THE not found, skip - Price Level
FAIR ISLE COFFEE CO @ LGMT FARM MRKT not found, skip - Index
CARNICERIA EL CERRITO not found, skip - Price Level
UTE CREEK GOLF COURSE not found, skip - Price Level
LEVY IPF 1 (MOBILE) not found, skip - Price Level
BOULDER BEEF (MOBILE PRE-PKG) not found, skip - Index
MOUNTAIN VIEW GRILL 2  (MOBILE) not found, skip - Index
CUPPA HOUSE COFFEE & TEA not found, skip - Price Level
(CU) C4C THE BAKERY not found, skip - Price Level
AREPA JOINT THE (MOBILE) not found, skip - Price Level
NIWOT WHEEL WORKS not found, skip - Price Level
ARAPAHOE MEAT CO not found, skip - Price Level
CHOP 5 SALAD KTICHEN not found, skip - Price Level
SMOKIN BOWLS not found, skip - Price Level
(CU)  FARRAND ALLEY not found, skip - Price Level
EGGLOO DELIGHTS (SPECIAL EVENT) not found, skip - Index
LA PUPUSA LOCA (MOBILE) not found, skip - Price Level
LA FAMILIA (MOBILE) not found, skip - Price Level
FRASIER MEADOWS MANOR KITCHEN not found, skip - Price Level
HOMEW

JAMESTOWN MERCANTILE CAFE not found, skip - Index
FLIGHT DECK GRILL (MOBILE) not found, skip - Price Level
COURTYARD BY MARRIOTT 1067 not found, skip - Price Level
CINEBARRE not found, skip - Price Level
SPIRIT HOUND DISTILLERS not found, skip - Price Level
JOHN HAROLD BUCKLEY POST 32 not found, skip - Price Level
LOAF N JUG 95 not found, skip - Price Level
HOLIDAY INN EXPRESS not found, skip - Price Level
NIWOT MARKET not found, skip - Price Level
KING SOOPERS 135 not found, skip - Price Level
BOULDER COUNTY JAIL not found, skip - Price Level
JFE SUSHI (KS-80) not found, skip - Price Level
BOULDER COUNTRY CLUB SNACK BAR not found, skip - Price Level
MOUNTAIN PEOPLES COOP not found, skip - Price Level
NATURAL GROCERS BY VITAMIN COTTAGE not found, skip - Price Level
GREEN BELLY FOODS BLDR (MOBILE PRE-PKG) not found, skip - Index
RIVERSIDE not found, skip - Price Level
BLUE OWL BOOKS not found, skip - Price Level
UCAR not found, skip - Price Level
ELDORADO CORNER MARKET not found, skip -

HILTON GARDEN INN - BOULDER not found, skip - Price Level
INN BETWEEN THE (PANTRY) not found, skip - Index
SIMPLY BULK MARKET not found, skip - Price Level
FALAFEL KING not found, skip - Price Level
SCRATCH (INSIDE 300 SUNS BREWING) not found, skip - Price Level
SPRUCE CAFE not found, skip - Price Level
LAS X AMERICAS TORTILLERIA Y CARNICERIA not found, skip - Price Level
SAVANNAH BEE COMPANY not found, skip - Price Level
PHO HUONG VIET not found, skip - Price Level
SWEET DOODLES FROYO not found, skip - Price Level
LONGS PEAK GROCERY MARKET not found, skip - Price Level
AMPERSAND COFFEE ROASTER not found, skip - Price Level
CARIBOU ROOM THE not found, skip - Price Level
HARVEST OF HOPE PANTRY not found, skip - Price Level
BOULDER MARRIOTT not found, skip - Price Level
LE FRIGO not found, skip - Price Level
PEARL STREET GRILL (COMMISSARY) not found, skip - Price Level
COURTYARD LOUISVILLE MARRIOTT not found, skip - Price Level
GROWING GARDENS not found, skip - Price Level
EMBASSY SUITES

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,OPERATIONAL,Roasted Toad,4.4,0.0,79.0
1,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163077,-105.098557,0,0,0.0,0.0,0.0
2,FA0003618,CARNICERIA EL CERRITO,936 MAIN ST LONGMONT CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.175377,-105.102119,OPERATIONAL,Carniceria El Cerrito,3.9,0.0,71.0
3,FA0000878,UTE CREEK GOLF COURSE,2000 UTE CREEK DR LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,40.189249,-105.063881,OPERATIONAL,Las Palmeras Express,4.5,0.0,87.0
4,FA0004983,LEVY IPF 1 (MOBILE),2400 COLORADO AVE BOULDER CO USA,MOBILE UNIT FULL FOOD SERVICE,MOBILE UNITS,40.009191,-105.266914,OPERATIONAL,Levy Restaurants,3.8,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
348,FA0005402,TIP TOP SAVORY PIES LLC,105-A N PUBLIC RD Lafayette CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,39.998682,-105.091059,OPERATIONAL,Tip Top Savory Pies,4.8,2.0,334.0
349,FA0004970,MENTAL HEALTH PARTNERS RYAN WELLNESS CENTER,1000 ALPINE AVE BOULDER CO USA,RESTAURANT 0 TO 100 SEATS,DAY TREATMENT PROGRAM,40.025323,-105.284209,OPERATIONAL,Mental Health Boulder County,2.1,2.0,23.0
350,FA0005473,ALTAVITA INDEPENDENT LIVING,1002 ALTA VITA CT LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,RESIDENTIAL FACILITIES,40.149471,-105.136453,OPERATIONAL,AltaVita Independent Living,4.1,2.0,13.0
351,FA0005424,BIG LOTS #4685,2151 MAIN ST Longmont CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.197288,-105.102622,OPERATIONAL,Big Lots,4.2,2.0,253.0


# Clean New Data 

In [21]:
#display the df
invalid_facilities = invalid_geocoded_df
invalid_facilities

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,OPERATIONAL,Roasted Toad,4.4,0.0,79.0
1,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163077,-105.098557,0,0,0.0,0.0,0.0
2,FA0003618,CARNICERIA EL CERRITO,936 MAIN ST LONGMONT CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.175377,-105.102119,OPERATIONAL,Carniceria El Cerrito,3.9,0.0,71.0
3,FA0000878,UTE CREEK GOLF COURSE,2000 UTE CREEK DR LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,40.189249,-105.063881,OPERATIONAL,Las Palmeras Express,4.5,0.0,87.0
4,FA0004983,LEVY IPF 1 (MOBILE),2400 COLORADO AVE BOULDER CO USA,MOBILE UNIT FULL FOOD SERVICE,MOBILE UNITS,40.009191,-105.266914,OPERATIONAL,Levy Restaurants,3.8,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
348,FA0005402,TIP TOP SAVORY PIES LLC,105-A N PUBLIC RD Lafayette CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,39.998682,-105.091059,OPERATIONAL,Tip Top Savory Pies,4.8,2.0,334.0
349,FA0004970,MENTAL HEALTH PARTNERS RYAN WELLNESS CENTER,1000 ALPINE AVE BOULDER CO USA,RESTAURANT 0 TO 100 SEATS,DAY TREATMENT PROGRAM,40.025323,-105.284209,OPERATIONAL,Mental Health Boulder County,2.1,2.0,23.0
350,FA0005473,ALTAVITA INDEPENDENT LIVING,1002 ALTA VITA CT LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,RESIDENTIAL FACILITIES,40.149471,-105.136453,OPERATIONAL,AltaVita Independent Living,4.1,2.0,13.0
351,FA0005424,BIG LOTS #4685,2151 MAIN ST Longmont CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.197288,-105.102622,OPERATIONAL,Big Lots,4.2,2.0,253.0


In [24]:
# Drop the cells with 0 in "Biz Status"
now_valid_facilities = invalid_facilities.loc[invalid_facilities["rating"] != 0]
now_valid_facilities

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,39.963180,-105.509594,OPERATIONAL,Roasted Toad,4.4,0.0,79.0
2,FA0003618,CARNICERIA EL CERRITO,936 MAIN ST LONGMONT CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,40.175377,-105.102119,OPERATIONAL,Carniceria El Cerrito,3.9,0.0,71.0
3,FA0000878,UTE CREEK GOLF COURSE,2000 UTE CREEK DR LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FAST FOOD LIMITED MENU,40.189249,-105.063881,OPERATIONAL,Las Palmeras Express,4.5,0.0,87.0
4,FA0004983,LEVY IPF 1 (MOBILE),2400 COLORADO AVE BOULDER CO USA,MOBILE UNIT FULL FOOD SERVICE,MOBILE UNITS,40.009191,-105.266914,OPERATIONAL,Levy Restaurants,3.8,0.0,5.0
7,FA0000100,CUPPA HOUSE COFFEE & TEA,1102 FRANCIS ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.177944,-105.116028,OPERATIONAL,Boba Kulture,4.7,0.0,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...
347,FA0005414,EFAA,1575 YARMOUTH AVE BOULDER CO USA,NO FEE LICENSE K12 SCHOOLS NON PROFIT,FOOD BANK,40.058472,-105.277708,OPERATIONAL,Emergency Family Assistance Association,4.2,2.0,49.0
348,FA0005402,TIP TOP SAVORY PIES LLC,105-A N PUBLIC RD Lafayette CO USA,GROCERY STORE 0 TO 15000 SQ FT,GROCERY FINISHED FOODS,39.998682,-105.091059,OPERATIONAL,Tip Top Savory Pies,4.8,2.0,334.0
349,FA0004970,MENTAL HEALTH PARTNERS RYAN WELLNESS CENTER,1000 ALPINE AVE BOULDER CO USA,RESTAURANT 0 TO 100 SEATS,DAY TREATMENT PROGRAM,40.025323,-105.284209,OPERATIONAL,Mental Health Boulder County,2.1,2.0,23.0
350,FA0005473,ALTAVITA INDEPENDENT LIVING,1002 ALTA VITA CT LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,RESIDENTIAL FACILITIES,40.149471,-105.136453,OPERATIONAL,AltaVita Independent Living,4.1,2.0,13.0


In [28]:
valid_facilities = now_valid_facilities[['facilityId', 'biz_status', 'rating', 'price_level', 'total_user_ratings']]
valid_facilities

,facilityId,biz_status,rating,price_level,total_user_ratings
0,FA0004494,OPERATIONAL,4.4,0.0,79.0
2,FA0003618,OPERATIONAL,3.9,0.0,71.0
3,FA0000878,OPERATIONAL,4.5,0.0,87.0
4,FA0004983,OPERATIONAL,3.8,0.0,5.0
7,FA0000100,OPERATIONAL,4.7,0.0,134.0
...,...,...,...,...,...
347,FA0005414,OPERATIONAL,4.2,2.0,49.0
348,FA0005402,OPERATIONAL,4.8,2.0,334.0
349,FA0004970,OPERATIONAL,2.1,2.0,23.0
350,FA0005473,OPERATIONAL,4.1,2.0,13.0


# Export Data

In [29]:
# Export unique address dataset w/ Google Data
output_data = "db_facility_data_w_ratings_2.csv"
valid_facilities.to_csv(output_data,header=True,index=False)